[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1AS2QNDgOgyl6k04v3Mt0Jxmmem4Cf3Ov?usp=sharing)
[![View Github](https://img.shields.io/badge/View%20Article-blue)](https://github.com/Shriniwas410/RAG-Appointment-Assistant)



In [7]:
!pip install datasets pandas pymongo sentence_transformers
!pip install -U transformers
# Install below if using GPU
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.8 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import requests
import json
import pandas as pd
import hashlib
import os
import pymongo
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

ip = requests.get('https://api.ipify.org').content.decode('utf8')
print('Add this IP address to MongoDB: {}'.format(ip))

os.environ["HF_TOKEN"] = "<your_huggingface_token>"
mongo_uri = "mongodb+srv://<username>:<password>@cluster0.np9dmzw.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"

# Connect to MongoDB
mongo_client = pymongo.MongoClient(mongo_uri)

db = mongo_client['appointment_finder']
collection = db['dps_appointments']

# Initialize tokenizer and model for natural language processing
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

# Initialize embedding model for vector search
embedding_model = SentenceTransformer("thenlper/gte-large")

# Create a session object
session = requests.Session()
# Simulated storage of hashed API keys
API_KEYS = {
    "key1": hashlib.sha256("secret_key_1".encode()).hexdigest(),
    "key2": hashlib.sha256("secret_key_2".encode()).hexdigest(),
}

# Headers as specified in the requirements
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "en-US,en;q=0.9",
    "Content-Type": "application/json;charset=UTF-8",
    "Origin": "https://public.txdpsscheduler.com",
    "Referer": "https://public.txdpsscheduler.com/",
    "Sec-Ch-Ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Google Chrome";v="122"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
}

# Make the POST request using the session object
cookies = {
    'ARRAffinity': 'e2c634607e44851e81f065fce3b73507fbe50f2156fd569962cb7167b11f16b9',
    'ARRAffinitySameSite': 'e2c634607e44851e81f065fce3b73507fbe50f2156fd569962cb7167b11f16b9'
}


In [3]:
# Function to insert data into MongoDB
def insert_data_mongo(zipcode):
    """
    Inserts the given DataFrame into the MongoDB collection.
    Clears the collection before insertion to ensure fresh data.

    Args:
    df (DataFrame): The DataFrame containing data to insert.
    """
    location_data, location_details = get_location_ids(zipcode)
    all_appointments = transform(location_data, location_details)
    all_appointments["embedding"] = all_appointments["AvailabilityDate"].apply(get_embedding)
    collection.delete_many({})
    documents = all_appointments.to_dict('records')
    collection.insert_many(documents)
    #inserting data into mongo
    print("Data ingestion into MongoDB completed")

In [4]:
# Function to check location availability
def location_check(locationId):
    """
    Checks the availability of appointments for a given location ID.

    Args:
    locationId (int): The ID of the location to check.
    api_key (str): The API key for authentication.

    Returns:
    dict or None: The response data if successful, None otherwise.
    """
    location_url = "https://publicapi.txdpsscheduler.com/api/AvailableLocationDates"

    data = {"LocationId": locationId, "TypeId": 81, "SameDay": False, "StartDate": None, "PreferredDay": 0}
    response = session.post(location_url, headers=headers, data=json.dumps(data), cookies=cookies)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        return None

def get_location_ids(zipcode):
    availablity_url = "https://publicapi.txdpsscheduler.com/api/AvailableLocation"
    # The body of the POST request. Adjust according to the API's requirements.
    data = {
        "TypeId": 81,
        "ZipCode": f"{zipcode}",
        "CityName": "",
        "PreferredDay": 0
    }
    response = session.post(availablity_url, headers=headers, data=json.dumps(data), cookies=cookies)
    data = response.json()
    location_details = []
    rows = []
    for location in data:
        location_info = {key: location.get(key, 'N/A') for key in ('Id', 'Name', 'Address', 'MapUrl')}
        location_details.append(location_info)
        location_data = location_check(location['Id'])
        rows.append(location_data)
    return rows, location_details


In [5]:
def transform(rows, location_details):
    df_data=pd.DataFrame()
    for data in rows:
        # Transforming the data
        transformed_data = [
            {
                'LocationId': location['LocationId'],
                'AvailabilityDate': location['FormattedAvailabilityDate'],
                'AvailabilityTime': ', '.join([slot['FormattedTime'] for slot in location['AvailableTimeSlots']]),
                'DayOfWeek': location['DayOfWeek'],
                'SlotId': ', '.join([str(slot['SlotId']) for slot in location['AvailableTimeSlots']])
            }
            for location in data['LocationAvailabilityDates']
        ]

        # Creating the DataFrame
        df = pd.DataFrame(transformed_data, columns=['LocationId', 'AvailabilityDate', 'AvailabilityTime', 'DayOfWeek', 'SlotId'])
        df_data = pd.concat([df_data, df], ignore_index=True)
    # Convert location_details to DataFrame
    df_locations = pd.DataFrame(location_details)

    # Merge the DataFrames on 'LocationId' from df_data and 'Id' from df_locations
    df_combined = pd.merge(df_data, df_locations, left_on='LocationId', right_on='Id', how='left')

    # Drop the 'Id' column as it's redundant with 'LocationId'
    df_combined.drop('Id', axis=1, inplace=True)

    # Reorder columns to have 'Name' first
    columns_order = ['Name', 'LocationId', 'AvailabilityDate', 'AvailabilityTime', 'DayOfWeek', 'SlotId', 'Address', 'MapUrl']
    df_combined = df_combined[columns_order]

    return df_combined


Connection to MongoDB successful


In [7]:
def get_embedding(text: str) -> list[float]:
    """
    Generates an embedding for the given text using the SentenceTransformer model.

    Args:
    text (str): The text to generate an embedding for.

    Returns:
    list: The generated embedding as a list of floats.
    """
    if not text.strip():
        print("Attempted to get embedding for empty text.")
        return []

    embedding = embedding_model.encode(text)

    return embedding.tolist()

<module 'torch.version' from '/usr/local/lib/python3.10/dist-packages/torch/version.py'>


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
def vector_search(user_query, collection):
    """
    Perform a vector search in the MongoDB collection based on the user query.

    Args:
    user_query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    list: A list of matching documents.
    """

    # Generate embedding for the user query
    query_embedding = get_embedding(user_query)

    if query_embedding is None:
        return "Invalid query or embedding generation failed."

    # Define the vector search pipeline
    vector_search_stage = {
        "$vectorSearch": {
            "index": "vector_index",
            "queryVector": query_embedding,
            "path": "embedding",
            "numCandidates": 150,  # Number of candidate matches to consider
            "limit": 10  # Return number of matches
        }
    }

    unset_stage = {
        "$unset": "embedding"  # Exclude the 'embedding' field from the results
    }

    project_stage = {
        "$project": {
                "_id": 0,  # Exclude the _id field
                "Name": 1,
                "LocationId": 1,  # Include the field
                "AvailabilityDate": 1,
                "AvailabilityTime": 1,  # Include the field
                "DayOfWeek": 1,  # Include the field
                "SlotId": 1,
                "Address": 1,
                "MapUrl":1,
                "score": {"$meta": "vectorSearchScore"}
        }
    }

    pipeline = [vector_search_stage, unset_stage, project_stage]

    # Execute the search
    results = collection.aggregate(pipeline)
    return list(results)

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [9]:
def get_search_result(query, collection):
    """
    Fetches search results for a given query from the MongoDB collection.
    This function uses the vector_search function to perform the search.

    Args:
    query (str): The user's query string.
    collection (MongoCollection): The MongoDB collection to search.

    Returns:
    str: A formatted string containing the search results.
    """
    get_knowledge = vector_search(query, collection)

    search_result = ""
    for result in get_knowledge:
        search_result += f"Name: {result.get('Name', 'N/A')}, LocationId: {result.get('LocationId', 'N/A')}, AvailabilityDate: {result.get('AvailabilityDate', 'N/A')}, AvailabilityTime: {result.get('AvailabilityTime', 'N/A')}, DayOfWeek: {result.get('DayOfWeek', 'N/A')}, SlotId: {result.get('SlotId', 'N/A')}, Address: {result.get('Address', 'N/A')}, MapUrl: {result.get('MapUrl', 'N/A')}\n"
    return search_result

In [28]:
def generate_answer(query):
    """
    Generates an answer to the user's query by combining it with search results
    and using the language model to generate a response.

    Args:
    query (str): The user's query.

    Returns:
    str: The generated answer to the query.
    """
  source_information = get_search_result(query, collection)
  combined_information = f"You a appointment assistant and you only answer queries realted to the Dps appointment, if any other questionis asked you will responsd with 'Not a valid query, Please ask question related to appointments. User query - {query}\nContinue to answer the query by using the Search Results:\n{source_information}."
  chat = [{"role": "user", "content": combined_information}]
  formatted_prompt = tokenizer.apply_chat_template(
      chat,
      tokenize=False,
      add_generation_prompt=True,
  )
  input_ids = tokenizer(combined_information, return_tensors="pt").to("cuda")
  response = model.generate(**input_ids, max_new_tokens=100000)
  response = tokenizer.decode(response[0], skip_special_tokens=False)
  response = response[len(formatted_prompt) :]  # remove input prompt from reponse
  response = response.replace("<eos>", "")  # remove eos token
  return response


In [29]:
def main():
    """
    Main function to run the appointment assistant.
    It ingests data into MongoDB, takes a user query, and prints the generated answer.
    """
    zipcode = "78758"
    insert_data_mongo(zipcode)
    query = "is there appointment available afternoon"
    print (generate_answer(query))

# Run the app
if __name__ == '__main__':
    main()

available at the Pflugerville Mega Center on Friday at 2:00 PM.
